In [1]:
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

torch.cuda.set_device(5)
device = torch.device('cuda')

device = "cuda" if torch.cuda.is_available() else "cpu"
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained(
    "Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16
)

model.to(device)

def BLIP_predict(img_pth):

    # url = "http://images.cocodataset.org/val2017/000000039769.jpg"
    # image = Image.open(requests.get(url, stream=True).raw)
    # img_pth='images/download_imgs/buildings.jpg'

    image = Image.open(img_pth).convert("RGB")

    inputs = processor(images=image, return_tensors="pt").to(device, torch.float16)

    generated_ids = model.generate(**inputs)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    print(generated_text)
    return generated_text

[2023-09-08 10:34:28,676] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
file_path = 'images/gnochi_mirror.jpeg'

generated_text = BLIP_predict(file_path)



a cat sitting on a wooden counter looking at itself in the mirror


/home-local/kwang-local/miniconda3/envs/floydkai/lib/python3.11/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# Use BLIP to create prompt text files.

## for Plug-and-Play-datasets, in PnP

In [ ]:
# img_pth = 'images/Plug-and-Play-datasets/wild-ti2i/data/bear.jpeg'
import os
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']

with open('prompt_Plug-and-Play-datasets.txt', 'w') as file:
    for root, dirs, files in os.walk('images/Plug-and-Play-datasets/'):
        for filename in files:
            file_path = os.path.join(root, filename)
            print(file_path)
            # 获取文件的扩展名
            _, extension = os.path.splitext(filename)
            if extension.lower() in image_extensions:
                generated_text = BLIP_predict(file_path)
                file.write(file_path + ': ' + generated_text + '\n')



## For pix2pix-zero's cats and dogs images.

In [ ]:

import os
image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff']

with open('prompt_pix2pix-zero.txt', 'w') as file:
    for root, dirs, files in os.walk('images/pix2pix-zero/'):
        for filename in files:
            file_path = os.path.join(root, filename)
            print(file_path)
            # 获取文件的扩展名
            _, extension = os.path.splitext(filename)
            if extension.lower() in image_extensions:
                generated_text = BLIP_predict(file_path)
                file.write(file_path + ': ' + generated_text + '\n')

